In [ ]:
x.scrape("https://www.zoopla.co.uk/for-sale/details/61532908/?search_identifier=8e874acb74a4f59f031ebea765f71b5a")

In [ ]:
x.quit_scrape()

In [ ]:
x.lat_long_data

In [ ]:
x.

In [ ]:
x.scrape("https://www.zoopla.co.uk/for-sale/details/48598857/?search_identifier=8e874acb74a4f59f031ebea765f71b5a")

## IMPORTS

In [1]:

#web scraping
import requests
import bs4
from bs4 import BeautifulSoup

#plotting, data analysis
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import seaborn as sns


#Dealing with CSVs from scrape
import csv
import subprocess
import os
from os.path import exists, isfile, join
from os import listdir
import glob

#scraping dependencies
import time 
from time import sleep
import re
import random
from pprint import pprint
from math import ceil
from collections import defaultdict

#my selenium model dependencies
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import selenium_code.selenium_scraper
from selenium.webdriver.chrome.options import Options
import selenium.webdriver.safari
from webdriver_manager.chrome import ChromeDriverManager

#import my selenium module
from selenium_code.selenium_scraper import ZooplaPageDriver, USER_AGENTS

## SCRAPING VARIABLES

## SCRAPING FUNCTIONS

### VPN Switcher Function

In [2]:
def connect_2_random_vpn():
    """Connect to a random new vpn"""
    print('There was an issue with scraping', '\n\n', 'Switching VPN location...','\n')
    with subprocess.Popen(["ivpn", "servers"], stdout=subprocess.PIPE) as proc:
        #print(proc.stdout.read().decode())
        vpns_stdouts = proc.stdout.read().decode()
    vpns = [vpn_name if index % 5 == 1 else None for index, vpn_name in enumerate(vpns_stdouts.split('|')) if index > 5]
    vpns = [item.strip() for item in vpns if item != None]
    selected_vpn = str(random.choice(vpns))
    with subprocess.Popen(["ivpn", "connect", selected_vpn], stdout=subprocess.PIPE) as proc:
        print(proc.stdout.read().decode())
    for wait_time in range(2,0,-1):
        print(f'Wait: {wait_time} seconds remaining')
        sleep(1)
    print('0 seconds remaining')
    print(f'\nContinuing with scrape...')

### Extraction function

There was an issue with scraping 

 Switching VPN location... 

[WireGuard] Connecting to: Holon, Tel Aviv, IL (Israel) il.wg.ivpn.net UDP:2049...
Connecting...
VPN                    : CONNECTED
                         il.wg.ivpn.net, Holon, Tel Aviv (IL), Israel
    Protocol           : WireGuard
    Local IP           : 172.17.63.113
    Server IP          : 185.191.207.197
    Connected          : 2022-06-04 17:44:55 +0100 BST
DNS                    : Default (auto)
Firewall               : Enabled
    Allow LAN          : false
    Allow IVPN servers : true

Tips:
 ivpn disconnect         Stop current VPN connection


Wait: 2 seconds remaining
Wait: 1 seconds remaining
0 seconds remaining

Continuing with scrape...


### NEED TO COOPT THIS CODE TO SCRAPE THE ZOOPLA PAGE!!!!

In [3]:
def make_zoopla_url(page_index, search_area):
    """create the URL for the next page"""
    return f"https://www.zoopla.co.uk/for-sale/property/london/?q={search_area}&results_sort=newest_listings&search_source=home&pn={page_index}"


def make_soup(url, headers):
    """make soup from passed URL"""
    r = requests.get(url, headers)
    return BeautifulSoup(r.text, 'html.parser')


def get_total_pages_for_query(soup, results_per_page):
    """identify number of pages to search through"""
    results_count_str = soup.find('p', attrs={'data-testid':"total-results"}).text
    listings_str = re.match(r"[0-9]+", results_count_str).group(0)
    listings = int(listings_str)
    return ceil(listings / results_per_page)


def zoopla_search_extractor(soup):
    """extract properties"""
    
    extraction_dict = {'address':[],
                       'price':[],
                       'bedrooms':[],
                       'bathrooms':[],
                       'lounges':[],
                       'url':[],
                       'nearby_station_1':[],
                       'nearby_station_2':[]
                      }
    
    
    property_listings = soup.find_all('div', attrs={"data-testid":re.compile(r"search-result_listing_")})
    
    for property in property_listings:
        
        
        #location
        try:
            address = property.find('p', attrs={"data-testid":"listing-description"}).text
        except:
            address = None
        extraction_dict['address'].append(address)

        
        #price
        try:
            price = property.find('div', attrs={"data-testid":re.compile(r"listing-price")} ).text
        except:
            price = None
        extraction_dict['price'].append(price)
        
        
        #bedrooms
        try:
            bedrooms = property.find('span', attrs={"data-testid":"bed"}).parent.next_sibling.text
        except:
            bedrooms = None
        extraction_dict['bedrooms'].append(bedrooms)
        
        #bathrooms        
        try:
            bathrooms = property.find('span', attrs={"data-testid":"bath"}).parent.next_sibling.text
        except:
            bathrooms = None
        extraction_dict['bathrooms'].append(bathrooms)
        
        #lounges
        try:
            lounges = property.find('span', attrs={"data-testid":"chair"}).parent.next_sibling.text
        except:
            lounges = None    
            
        extraction_dict['lounges'].append(lounges)
        
        #URL to listing page
        try:
            property_url = property.find('a', attrs={"data-testid":"listing-details-link"})['href']
            property_url_available = True
        except:
            property_url = None
            property_url_available = False
        extraction_dict['url'].append(property_url)
        
        #NEARBY LOCATION DATA
        try:
            nearby_locations = property.find('div', attrs={"data-testid":"listing-transport"})
            nearby_locations = [location.text for location in nearby_locations.children]
            
            try:
                extraction_dict['nearby_station_1'].append(nearby_locations[0])
            except:
                extraction_dict['nearby_station_1'].append(None)
            try:
                extraction_dict['nearby_station_2'].append(nearby_locations[1])
            except:
                extraction_dict['nearby_station_2'].append(None)
        except:
            extraction_dict['nearby_station_1'].append(None)
            extraction_dict['nearby_station_2'].append(None)
        
        
        
    return extraction_dict

### Page Scraper Function

In [4]:
def zoopla_all_listings_scraper(location, n_pages=1, delay=2):
    
    #Creating the dictionary which will be used to save our scraped data on each iteration before saving in .csv
    extraction = {'address':[],
                       'price':[],
                       'bedrooms':[],
                       'bathrooms':[],
                       'lounges':[],
                       'url':[],
                       'nearby_station_1':[],
                       'nearby_station_2':[]}

    
    
    
    #creating .csv files if they do not already exist. They will be appended to during the scrape
    #creating file
    file_location = "data/csv_data/" + "_".join(location.split()) + "_scrape_.csv"
    keys = extraction.keys()
    if not exists(file_location):
        with open(file_location, "w") as outfile:
            writer = csv.writer(outfile, delimiter = "\t")
            writer.writerow(keys)
    
    
    
    
    
    #Formatting the query and location strings to be usable in our URL
    # fmat_location= "%20".join(location.strip().split()).lower()    #place separated by %20
    # fmat_query= "%20".join(query.strip().split()).lower()          #listing title separated by %20
     
    
    
        
        
    #if all pages are to be scraped then get the number of pages
    # Keep trying by changing vpns until we have this result up to a limited maximum number of tries
    results_per_page = 25
    listing_num_scraped = False
    vpn_switch_counter = 0
    
    if n_pages == 'all':
        while not listing_num_scraped:
            try:

                url = make_zoopla_url(0, location)            
                header_choice = {"User-Agent":random.choice(USER_AGENTS)}
                r = requests.get(url)
                soup = BeautifulSoup(r.text, 'html.parser')

                n_pages = get_total_pages_for_query(soup, results_per_page)
                listing_num_scraped = True
            except:
                print("Couldn't scrape the number of listings")
                print("Attempting a different VPN")
                connect_2_random_vpn() #switches the VPN
                vpn_switch_counter += 1
                if vpn_switch_counter == 5:
                    break

    
    
    
    
    
    
    
    
    
    # TIME TO SCRAPE EACH PAGE!
    print(f"Commencing Scrape of {n_pages} pages")
    print(f"Scraping {n_pages * results_per_page} listings in {location.upper()}")

    print("!"*50)
    print("="*50)
    
    # SCRAPE FOR LOOP ####################################################################################
    
    total_captchas = 0
    for current_page_index in range(1, n_pages+1):
        
        print(f'Attempting to scrape page {current_page_index} of {n_pages}')
    
        current_page_url = make_zoopla_url(current_page_index, location)
        
        #Using requests to return json and then turning to soup using our defined soupify function
        
        possible_captcha = True
        captcha_counter = 0
        while possible_captcha: 
            #Set a limit on the number of times we try to switch VPN
            if captcha_counter == 5:
                break
            
            #Connect to the page using random headeres 
            header_choice = {"User-Agent":random.choice(USER_AGENTS)}
            r = requests.get(current_page_url, headers=header_choice)
            soup = BeautifulSoup(r.text, 'html.parser')
            
            #If we get CAPTCHA'd try a different VPN server and retry connecting. Increase captcha_counter by 1
            #If NOT CAPTCHA'd then break the while loop and continue
            #print(len(soup.get_text().lower()))
            if 'captcha' in soup.get_text().lower(): 
                captcha_counter += 1
                print("~o" * 5 + "CAPTCHA DETECTED!" + "o~" *5)
                connect_2_random_vpn()
            else:
                print('No CAPTCHA on page')
                possible_captcha = False
          
        
        
                
        
        # Attempt to extract all of the data from the page which we hope is not still CAPTCHA'd
        #zoopla_search_extractor(soup)
        for col_name, data in zoopla_search_extractor(soup).items():
            extraction[col_name].extend(data)
            print(f"{col_name}: {len([points for points in data if points != None])}")
        # Add any more captchas to the TOTAL CAPTCHAS
        total_captchas += captcha_counter
        
        # APPEND TO CSV any data extracted at certain intervals
        
        #when to save work to csv
        if current_page_index % 5 == 0 or current_page_index == n_pages:
            #Every n pages save the output
            with open(file_location, "a") as outfile:
                writer = csv.writer(outfile, delimiter = "\t")
                writer.writerows(zip(*[extraction[key] for key in keys]))
            
            #Empty the extraction variable for the next scrapes
            print(extraction)
            extraction = {key:[] for key in extraction.keys()}
            
            
            
        print(f"Number of items currently in extraction buffer:{len(extraction['address'])}")
        print(f"TOTAL CAPTCHAS DURING SCRAPE: {total_captchas}")
        print("=" * 30)
        
        #DELAY loading next page by preset amount
        sleep(delay)
        
    return extraction

In [ ]:
#zoopla_all_listings_scraper("london", n_pages='all', delay=2)

In [37]:
csv_uri = "./data/csv_data/london_scrape.csv"

df = pd.read_csv(csv_uri, sep='\t')


In [38]:
df

,address,price,bedrooms,bathrooms,lounges,url,nearby_station_1,nearby_station_2
0,"Park Street, Chelsea/London SW6","£1,375,000",2,2,1,/new-homes/details/61623573/?search_identifier...,0.1 miles Imperial Wharf,0.2 miles Chelsea Harbour Pier
1,"Bridges Court, London SW11","£440,000",1,1,1,/for-sale/details/61567435/?search_identifier=...,0.4 miles Chelsea Harbour Pier,0.4 miles Imperial Wharf
2,"Carshalton Grove, Sutton SM1","Guide price£650,000",3,2,2,/for-sale/details/61623414/?search_identifier=...,0.6 miles Carshalton,0.6 miles Sutton (London)
3,"Rutland Rd, Forest Gate E7","Guide price£545,000",3,1,2,/for-sale/details/61623411/?search_identifier=...,0.4 miles East Ham,0.5 miles Upton Park
4,"Rutland Rd, Forest Gate E7","Guide price£545,000",3,1,2,/for-sale/details/61623405/?search_identifier=...,0.4 miles East Ham,0.5 miles Upton Park
...,...,...,...,...,...,...,...,...
9995,"Burlington Road, Osterley, Isleworth TW7","£750,000",3,1,2,/for-sale/details/61476741/?search_identifier=...,0.3 miles Osterley,0.5 miles Hounslow East
9996,"Tufnell Park, Tufnell Park, London N7","£375,000",1,1,na,/for-sale/details/61476757/?search_identifier=...,0.5 miles Upper Holloway,0.5 miles Tufnell Park
9997,"Holmewood Gardens, Brixton Hill, London SW2","£700,000",2,1,na,/for-sale/details/61476758/?search_identifier=...,0.6 miles Streatham Hill,0.8 miles Tulse Hill
9998,"Coldershaw Road, Ealing W13","Guide price£535,000",2,1,1,/for-sale/details/61476750/?search_identifier=...,0.5 miles West Ealing,0.6 miles Drayton Green


### selenium extraction - Too slow for use in project

In [ ]:
"""selenium_extraction = {
                        'lat':[],
                       'long':[],
                       'first_listed_date':[],
                       'first_listed_text':[],
                       'first_listed_price':[],
                       'last_sold_date':[],
                       'last_sold_text':[],
                       'last_sold_price':[],
                       'tenure':[],
                       'council_tax_band':[]}




#tenure and council tax band to be added to selenium scrape
        tenure = 'na'
        extraction_dict['tenure'].append(tenure)
        council_tax_band = 'na'
        extraction_dict['council_tax_band'].append(council_tax_band)
        
        
        
        #use Selenium class to scrape each listing
        if property_url_available:
            try:
                scrape_url = "https://www.zoopla.co.uk" + property_url
                selenium_scrape_instance = ZooplaPageDriver()
                selenium_results = selenium_scrape_instance.scrape(scrape_url)
                for key, value in selenium_results.items():
                    extraction_dict[key].append(value)
  
                
            except:
                print('ERROR: Unable to scrape property url using selenium')"""

### Second page scraper for zoopla historical data

In [5]:
def find_iter(soup, tagname):
    tag = soup.find(tagname)
    while tag is not None:
        yield tag
        tag = tag.find_next(tagname)

In [6]:
def make_historical_zoopla_url(page_index, search_area):
    """create the URL for the next historical zoopla page"""
    return f"https://www.zoopla.co.uk/house-prices/london/?q={search_area}&search_source=house-prices&pn={page_index}"


In [96]:

    def extract_historical_page_data(soup):

        extract_columns = ['address', 'last_sold_date', 'last_sold_price', 'estimated_price', 'bedrooms', 'bathrooms', 'lounges', 'url', 'tenure', 'property_type' ]
        extraction = {key:[] for key in extract_columns}




        for li in soup.find('ul', attrs={'data-testid':'result-list'}):

            each_property = li.div


            try:
                if each_property.attrs['aria-disabled'] == 'true':
                    #property has been deactivated
                    print('property has been deactivated')
                    continue
            except:
                print('property missing aria-disabled attr')


            try:
                address = each_property.find('h2').text
            except:
                print('property missing h2 tag for address')


            last_sold_date = None
            last_sold_price = None
            estimated_price = None
            try:
                price_data = each_property.findChild('div',recursive=False).findChildren('div', recursive=False)
                for div in price_data:
                    if 'Last sold' in div.text:
                        last_sold_date = div.find('label').text
                        last_sold_price = div.findChildren('label', recursive=False)[1].text
                    elif 'Estimated price' in div.text:
                            estimated_price = div.findChildren('label', recursive=False)[1].text
            except:
                print('failed to get price_data')


            try:
                property_url = each_property.a['href']
            except:
                print('failed to get property_url')





            bedrooms, bathrooms, lounges = None, None, None
            tenure, property_type = None, None

            #used for the first div in additional info (there could be 0, 1 or 2 divs)
            try:
                additional_info_divs = each_property.a.div.findChildren('div', recursive=False)[1].findChildren('div', recursive=False)
                room_counts = additional_info_divs[0].children
                for div in room_counts:
                    if 'bed' in div.text.lower():
                        bedrooms = re.sub(r'[^0-9]', '', div.text)
                    elif 'bath' in div.text.lower():
                        bathrooms = re.sub(r'\D', '', div.text)
                    elif 'reception' in div.text.lower():
                        lounges = re.sub(r'\D', '', div.text)
                    else: #means there is only info on the tenure and property
                        try:
                            for each in div.findChildren('div', recursive=False):
                                if 'freehold' in each.text.lower():
                                    tenure = 'freehold'
                                elif 'leasehold' in each.text.lower():
                                    tenure = each.text.lower()
                                else:
                                    property_type = each.text.lower()
                        except:
                            print('failed to get tenure info')

            except:
                print('failed to get bed-bath-receiption info')


            #for the case where there are two div's in additional info
            try:
                property_descriptions = additional_info_divs[1].findChildren('div', recursive=False)

                for each in property_descriptions:

                    if 'freehold' in each.text.lower():
                        tenure = 'freehold'
                    elif 'leasehold' in each.text.lower():
                        tenure = each.text.lower()
                    else:
                        property_type = each.text.lower()

            except:
                pass


            extraction['address'].append(address)
            extraction['last_sold_date'].append(last_sold_date)
            extraction['last_sold_price'].append(last_sold_price)
            extraction['estimated_price'].append(estimated_price)
            extraction['bedrooms'].append(bedrooms)
            extraction['bathrooms'].append(bathrooms)
            extraction['lounges'].append(lounges)
            extraction['url'].append(property_url)
            extraction['tenure'].append(tenure)
            extraction['property_type'].append(property_type)

        return extraction

### Historical crawler

In [97]:
def zoopla_historical_listings_scraper(location, n_pages=1, delay=2):
    
    #Creating the dictionary which will be used to save our scraped data on each iteration before saving in .csv
    extract_columns = ['address', 'last_sold_date', 'last_sold_price', 'estimated_price', 'bedrooms', 'bathrooms', 'lounges', 'url', 'tenure', 'property_type' ]
    extraction = {key:[] for key in extract_columns}

    
    
    
    #creating .csv files if they do not already exist. They will be appended to during the scrape
    #creating file
    file_location = "data/csv_data/" + "_".join(location.split()) + "_historical_scrape.csv"
    keys = extraction.keys()
    if not exists(file_location):
        with open(file_location, "w") as outfile:
            writer = csv.writer(outfile, delimiter = "\t")
            writer.writerow(keys)
    
    
    
    
    
    #Formatting the query and location strings to be usable in our URL
    # fmat_location= "%20".join(location.strip().split()).lower()    #place separated by %20
    # fmat_query= "%20".join(query.strip().split()).lower()          #listing title separated by %20
     
    
    
        
        
    #if all pages are to be scraped then get the number of pages
    # Keep trying by changing vpns until we have this result up to a limited maximum number of tries
    results_per_page = 10
    listing_num_scraped = False
    vpn_switch_counter = 0
    
    if n_pages == 'all':
        while not listing_num_scraped:
            try:
                
                url = make_historical_zoopla_url(0, location)         
                header_choice = {"User-Agent":random.choice(USER_AGENTS)}
                r = requests.get(url)
                soup = BeautifulSoup(r.text, 'html.parser')
                for i in find_iter(soup, 'label'):
                    if 'properties found' in i.text:
                        property_total_str = i.text.replace(",","")
                        print(property_total_str)
                        historical_property_count = int(re.search(r"[\d]+", property_total_str).group(0))
                        print(historical_property_count)
                        break
                
                
                

                n_pages = int(ceil(historical_property_count / results_per_page))
                listing_num_scraped = True
            except:
                print("Couldn't scrape the number of listings")
                print("Attempting a different VPN")
                connect_2_random_vpn() #switches the VPN
                vpn_switch_counter += 1
                if vpn_switch_counter == 5:
                    break

    
    
    
    
    
    
    
    
    
    # TIME TO SCRAPE EACH PAGE!
    print(f"Commencing Scrape of {n_pages} pages")
    print(f"Scraping {historical_property_count} listings in {location.upper()}")

    print("!"*50)
    print("="*50)
    
    # SCRAPE FOR LOOP ####################################################################################
    
    total_captchas = 0
    for current_page_index in range(1090, n_pages+1):
        
        print(f'Attempting to scrape page {current_page_index} of {n_pages}')
    
        current_page_url = make_historical_zoopla_url(current_page_index, location)
        
        #Using requests to return json and then turning to soup using our defined soupify function
        
        possible_captcha = True
        captcha_counter = 0
        while possible_captcha: 
            #Set a limit on the number of times we try to switch VPN
            if captcha_counter == 5:
                break
            
            
            try:
                #Connect to the page using random headeres 
                header_choice = {"User-Agent":random.choice(USER_AGENTS)}
                r = requests.get(current_page_url, headers=header_choice)
            except:
                sleep(20)
                connect_2_random_vpn()
                captcha_counter += 1
            soup = BeautifulSoup(r.text, 'html.parser')
            
            #If we get CAPTCHA'd try a different VPN server and retry connecting. Increase captcha_counter by 1
            #If NOT CAPTCHA'd then break the while loop and continue
            #print(len(soup.get_text().lower()))
            
            for h1 in soup.find_all('h1'):
                if h1.text == "House prices in London":
                    possible_captcha = False
                    print('No CAPTCHA on page')
                    break
            if not possible_captcha:
                break
            print("~o" * 5 + "CAPTCHA DETECTED!" + "o~" *5)
            captcha_counter += 1
            connect_2_random_vpn()

            
            
            
            
          
        
        
                
        
        # Attempt to extract all of the data from the page which we hope is not still CAPTCHA'd
        for col_name, data in extract_historical_page_data(soup).items():
            extraction[col_name].extend(data)
            print(f"{col_name}: {len([points for points in data if points != None])}")
        # Add any more captchas to the TOTAL CAPTCHAS
        total_captchas += captcha_counter
        
        
        
        print(f"Number of items currently in extraction buffer:{len(extraction['address'])}")
        
        
        
        # APPEND TO CSV any data extracted at certain intervals
        
        #when to save work to csv
        if current_page_index % 5 == 0 or current_page_index == n_pages:
            #Every n pages save the output
            with open(file_location, "a") as outfile:
                writer = csv.writer(outfile, delimiter = "\t")
                writer.writerows(zip(*[extraction[key] for key in keys]))
            
            #Empty the extraction variable for the next scrapes
            
            extraction = {key:[] for key in extraction.keys()}
            print("Emptying buffer...number of addresses now in buffer:", len(extraction['address']))
                 
        
        print(f"TOTAL CAPTCHAS DURING SCRAPE: {total_captchas}")
        print("=" * 30)
        
        #DELAY loading next page by preset amount
        sleep(delay)
        
    return extraction

In [ ]:
zoopla_historical_listings_scraper("london", n_pages="all", delay=2)

In [ ]:
historical_data_uri = "./data/csv_data/london_historical_scrape.csv"




df = pd.read_csv(historical_data_uri, sep='\t')

### Repurposing the historical scraper to randomly select pages

In [ ]:
def zoopla_random_historical_listings_scraper(location, n_pages=1, delay=2):
    
    #Creating the dictionary which will be used to save our scraped data on each iteration before saving in .csv
    extract_columns = ['address', 'last_sold_date', 'last_sold_price', 'estimated_price', 'bedrooms', 'bathrooms', 'lounges', 'url', 'tenure', 'property_type' ]
    extraction = {key:[] for key in extract_columns}

    
    
    
    #creating .csv files if they do not already exist. They will be appended to during the scrape
    #creating file
    file_location = "data/csv_data/" + "_".join(location.split()) + "_random_historical_scrape.csv"
    keys = extraction.keys()
    if not exists(file_location):
        with open(file_location, "w") as outfile:
            writer = csv.writer(outfile, delimiter = "\t")
            writer.writerow(keys)
     
    
    
        
        
    #if all pages are to be scraped then get the number of pages
    # Keep trying by changing vpns until we have this result up to a limited maximum number of tries
    results_per_page = 10
    listing_num_scraped = False
    vpn_switch_counter = 0
    
    if n_pages == 'all':
        while not listing_num_scraped:
            try:
                
                url = make_historical_zoopla_url(0, location)         
                header_choice = {"User-Agent":random.choice(USER_AGENTS)}
                r = requests.get(url)
                soup = BeautifulSoup(r.text, 'html.parser')
                for i in find_iter(soup, 'label'):
                    if 'properties found' in i.text:
                        property_total_str = i.text.replace(",","")
                        print(property_total_str)
                        historical_property_count = int(re.search(r"[\d]+", property_total_str).group(0))
                        print(historical_property_count)
                        break
                
                
                

                n_pages = int(ceil(historical_property_count / results_per_page))
                listing_num_scraped = True
            except:
                print("Couldn't scrape the number of listings")
                print("Attempting a different VPN")
                connect_2_random_vpn() #switches the VPN
                vpn_switch_counter += 1
                if vpn_switch_counter == 5:
                    break

    
    
    
    
    
    
    
    
    
    # TIME TO SCRAPE EACH PAGE!
    print(f"Commencing Scrape of {n_pages} pages")
    print(f"Scraping {historical_property_count} listings in {location.upper()}")

    print("!"*50)
    print("="*50)
    
    # SCRAPE FOR LOOP ####################################################################################
    
    total_captchas = 0
    for current_page_index in random.sample(range(1090, n_pages+1):
        
        print(f'Attempting to scrape page {current_page_index} of {n_pages}')
    
        current_page_url = make_historical_zoopla_url(current_page_index, location)
        
        #Using requests to return json and then turning to soup using our defined soupify function
        
        possible_captcha = True
        captcha_counter = 0
        while possible_captcha: 
            #Set a limit on the number of times we try to switch VPN
            if captcha_counter == 5:
                break
            
            
            try:
                #Connect to the page using random headeres 
                header_choice = {"User-Agent":random.choice(USER_AGENTS)}
                r = requests.get(current_page_url, headers=header_choice)
            except:
                sleep(20)
                connect_2_random_vpn()
                captcha_counter += 1
            soup = BeautifulSoup(r.text, 'html.parser')
            
            #If we get CAPTCHA'd try a different VPN server and retry connecting. Increase captcha_counter by 1
            #If NOT CAPTCHA'd then break the while loop and continue
            #print(len(soup.get_text().lower()))
            
            for h1 in soup.find_all('h1'):
                if h1.text == "House prices in London":
                    possible_captcha = False
                    print('No CAPTCHA on page')
                    break
            if not possible_captcha:
                break
            print("~o" * 5 + "CAPTCHA DETECTED!" + "o~" *5)
            captcha_counter += 1
            connect_2_random_vpn()

            
            
            
            
          
        
        
                
        
        # Attempt to extract all of the data from the page which we hope is not still CAPTCHA'd
        for col_name, data in extract_historical_page_data(soup).items():
            extraction[col_name].extend(data)
            print(f"{col_name}: {len([points for points in data if points != None])}")
        # Add any more captchas to the TOTAL CAPTCHAS
        total_captchas += captcha_counter
        
        
        
        print(f"Number of items currently in extraction buffer:{len(extraction['address'])}")
        
        
        
        # APPEND TO CSV any data extracted at certain intervals
        
        #when to save work to csv
        if current_page_index % 5 == 0 or current_page_index == n_pages:
            #Every n pages save the output
            with open(file_location, "a") as outfile:
                writer = csv.writer(outfile, delimiter = "\t")
                writer.writerows(zip(*[extraction[key] for key in keys]))
            
            #Empty the extraction variable for the next scrapes
            
            extraction = {key:[] for key in extraction.keys()}
            print("Emptying buffer...number of addresses now in buffer:", len(extraction['address']))
                 
        
        print(f"TOTAL CAPTCHAS DURING SCRAPE: {total_captchas}")
        print("=" * 30)
        
        #DELAY loading next page by preset amount
        sleep(delay)
        
    return extraction

In [1]:
random.sample(range(1000000), 10)

NameError: name 'random' is not defined

In [73]:
df

,address,last_sold_date,last_sold_price,estimated_price,bedrooms,bathrooms,lounges,url,tenure,property_type
0,"1 Bickley Crescent, Bromley, BR1 2DN",Last sold - Oct 1999,"£97,500","£461,000 - £564,000",2.0,NaN,2.0,/property/uprn/100020390465/,freehold,semi-detached house
1,"1 Chadd Drive, Bromley, BR1 2DP",Last sold - Jul 2018,"£675,000","£789,000 - £872,000",NaN,NaN,NaN,/property/uprn/100020393770/,freehold,detached house
2,"1 Page Heath Lane, Bromley, BR1 2DR",Last sold - Jan 2007,"£480,000","£740,000 - £905,000",4.0,2.0,2.0,/property/uprn/100020408794/,freehold,semi-detached house
3,"1 Beresford Drive, Bromley, BR1 2DU",Last sold - Nov 2006,"£420,000","£830,000 - £918,000",3.0,1.0,3.0,/property/uprn/100020390386/,freehold,detached house
4,"1 St Michael's Close, Bromley, BR1 2DX",Last sold - Nov 2006,"£376,000","£740,000 - £904,000",NaN,NaN,NaN,/property/uprn/100020416127/,freehold,detached house
5,"1 Southborough Road, Bromley, BR1 2EA",Last sold - May 1998,"£185,000","£973,000 - £1,189,000",4.0,2.0,2.0,/property/uprn/100020415051/,freehold,detached house
6,"1 Lancaster Gardens, Bromley, BR1 2ED",Last sold - Feb 2002,"£375,000","£588,000 - £882,000",3.0,NaN,2.0,/property/uprn/10003617154/,freehold,mid terrace house
7,"1 Falcon Avenue, Bromley, BR1 2EH",Last sold - Oct 2009,"£571,000","£1,049,000 - £1,159,000",5.0,NaN,2.0,/property/uprn/100020397356/,freehold,detached house
8,"1 Parkside Avenue, Bromley, BR1 2EJ",Last sold,-,"£506,000 - £619,000",NaN,NaN,NaN,/property/uprn/100020409903/,freehold,detached house
9,"1 Sunningdale Road, Bromley, BR1 2EU",Last sold,-,"£718,000 - £878,000",NaN,NaN,NaN,/property/uprn/100020416607/,NaN,detached house


## CREATING DATAFRAMES

In [ ]:
# Get CSV files list from a folder
path = os.getcwd()
csv_files = glob.glob(path + "data/csv_data/*.csv")

#Could have separated these during the scrape but we separate the files by which country they are in...
#This will help when we are tidying up our salary data and adding city columns
usa_csv_files = [file_name for file_name in csv_files for city in usa_cities if "/{}_scrape.csv".format(city.replace(" ", "_")) in file_name]
uk_csv_files = [file_name for file_name in csv_files for city in uk_cities if "/{}_scrape.csv".format(city.replace(" ", "_")) in file_name]

# Read each CSV file into DataFrame
# This creates a list of dataframes

df_list = []
for file in usa_csv_files:
    current_df = pd.read_csv(file, sep='\t')
    current_city = [city for city in usa_cities if "/{}_scrape.csv".format(city.replace(" ", "_")) in file][0]
    current_df['city'] = current_city
    current_df['country'] = 'usa'
    df_list.append(current_df)
    
for file in uk_csv_files:
    current_df = pd.read_csv(file, sep='\t')
    current_city = [city for city in uk_cities if "/{}_scrape.csv".format(city.replace(" ", "_")) in file][0]
    current_df['city'] = current_city
    current_df['country'] = 'uk'
    df_list.append(current_df)
    

#df_list = (pd.read_csv(file, sep='\t') for file in csv_files)

# Concatenate all DataFrames

df = pd.concat(df_list, ignore_index=True)


